In [45]:
from pyspark.sql import SparkSession
import re

# (8 cores, 16gb per machine) x 5 = 40 cores

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.87:7077") \
        .appName("mei_wu_part-1")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .getOrCreate()
#        .config('spark.executor.cores', 2)\


# Old API (RDD)
spark_context = spark_session.sparkContext

rdd_en = spark_context.newAPIHadoopFile(
    'hdfs://192.168.2.87:9000/europarl/europarl-v7.sv-en.en',
    'org.apache.hadoop.mapreduce.lib.input.TextInputFormat',
    'org.apache.hadoop.io.LongWritable',
    'org.apache.hadoop.io.Text'
)\
.cache() # Keep this RDD in memory!

rdd_sv = spark_context.newAPIHadoopFile(
    'hdfs://192.168.2.87:9000/europarl/europarl-v7.sv-en.sv',
    'org.apache.hadoop.mapreduce.lib.input.TextInputFormat',
    'org.apache.hadoop.io.LongWritable',
    'org.apache.hadoop.io.Text'
)\
.cache() # Keep this RDD in memory!

In [46]:
en_count = rdd_en.count() # A.1.1 Counts the lines in English
en_count

1862234

In [47]:
sv_count = rdd_sv.count() # A.1.2 Counts the lines in Swedish
sv_count

1862234

In [48]:
en_count == sv_count # A.1.3 Verify the lines are same in both languages

True

In [49]:
rdd_en.getNumPartitions() # A.1.4 Count number of partitions in English

2

In [50]:
rdd_sv.getNumPartitions() # A.1.4 Count number of partitions in Swedish

3

In [72]:
# A.2.1

def lowerNsplit(w):
    w = w.lower().split(' ')
    return w

# A.2.2
rdd_en.mapValues(lowerNsplit).take()

[(0, ['resumption', 'of', 'the', 'session']),
 (26,
  ['i',
   'declare',
   'resumed',
   'the',
   'session',
   'of',
   'the',
   'european',
   'parliament',
   'adjourned',
   'on',
   'friday',
   '17',
   'december',
   '1999,',
   'and',
   'i',
   'would',
   'like',
   'once',
   'again',
   'to',
   'wish',
   'you',
   'a',
   'happy',
   'new',
   'year',
   'in',
   'the',
   'hope',
   'that',
   'you',
   'enjoyed',
   'a',
   'pleasant',
   'festive',
   'period.']),
 (234,
  ['although,',
   'as',
   'you',
   'will',
   'have',
   'seen,',
   'the',
   'dreaded',
   "'millennium",
   "bug'",
   'failed',
   'to',
   'materialise,',
   'still',
   'the',
   'people',
   'in',
   'a',
   'number',
   'of',
   'countries',
   'suffered',
   'a',
   'series',
   'of',
   'natural',
   'disasters',
   'that',
   'truly',
   'were',
   'dreadful.']),
 (426,
  ['you',
   'have',
   'requested',
   'a',
   'debate',
   'on',
   'this',
   'subject',
   'in',
   'the',
   'c

In [73]:
# A.2.2
rdd_sv.mapValues(lowerNsplit).take(10)

[(0, ['återupptagande', 'av', 'sessionen']),
 (29,
  ['jag',
   'förklarar',
   'europaparlamentets',
   'session',
   'återupptagen',
   'efter',
   'avbrottet',
   'den',
   '17',
   'december.',
   'jag',
   'vill',
   'på',
   'nytt',
   'önska',
   'er',
   'ett',
   'gott',
   'nytt',
   'år',
   'och',
   'jag',
   'hoppas',
   'att',
   'ni',
   'haft',
   'en',
   'trevlig',
   'semester.']),
 (212,
  ['som',
   'ni',
   'kunnat',
   'konstatera',
   'ägde',
   '"den',
   'stora',
   'år',
   '2000-buggen"',
   'aldrig',
   'rum.',
   'däremot',
   'har',
   'invånarna',
   'i',
   'ett',
   'antal',
   'av',
   'våra',
   'medlemsländer',
   'drabbats',
   'av',
   'naturkatastrofer',
   'som',
   'verkligen',
   'varit',
   'förskräckliga.']),
 (409,
  ['ni',
   'har',
   'begärt',
   'en',
   'debatt',
   'i',
   'ämnet',
   'under',
   'sammanträdesperiodens',
   'kommande',
   'dagar.']),
 (488,
  ['till',
   'dess',
   'vill',
   'jag',
   'att',
   'vi,',
   'som',
   '

In [74]:
# A.2.3 Verify that the line counts still match after the pre-processing.
rdd_en.mapValues(lowerNsplit).count() == rdd_sv.mapValues(lowerNsplit).count()

True

In [ ]:
# A.3.1 Use Spark to compute the 10 most frequently according words in the English language corpus. Repeat for the other language.


In [35]:
spark_session.stop()